# The Python Package for Maximum Likelihood Estimates for Variance
# (<span style="color:purple;"> MLEV </span>)

## Introduction
### Goal
Implement the most recently published variance estimator for _linear models_

### What
Variance of the random error term ($\sigma^2$)
$$ y = X \beta + \epsilon $$
$ \epsilon $ is from $ i.i.d.N(0,\sigma^2) $

### Why
- Used in model comparison
- Given the size of the model, pick the model with lowest variance estmiates, which explain the most of variance

### How
- Least square estimator
  - works for low dimension data ($n>p$)
  - does not exist in high dimension data ($n<p$)
- Maximum Likelyhood Estimator
  - works for both low and high dimensions
  - more accurate compared to LS estimator in low dimensions


## Implement of MLEV

### Python code

In [2]:
# Load pre-requisite packages
import numpy as np
import scipy.optimize as optimx

# Define functions within the MLEV class
class MLEV:
    
    def __init__(self, data):
        self.y = data['y']
        self.X = data['X']
        self.n = float(data['X'].shape[0])
        self.p = float(data['X'].shape[1])
        self.theta2_init = float(1)

    def describe(self):
        print('The dimension of design matrix is (' + str(self.n) + ', ' + str(self.p) + ')')
        return(self)

    def eigen(self):
        self.XXt = np.dot(self.X, self.X.T)
        self.lbd, self.vec = np.linalg.eigh(self.XXt)
        self.yTilde = np.dot(self.vec.T, self.y)
        self.yTildeSq = self.yTilde**2
        return(self)

    def mlevObj(self, theta2):
        out1 = np.log(np.sum(self.yTildeSq/(theta2/self.p*self.lbd+1.0))) + 1.0/self.n*np.sum(np.log(theta2/self.p*self.lbd+1.0))
        return(out1)

    def getTheta2(self, theta2_init=5.0):
        self.theta2_est = optimx.fmin_l_bfgs_b(func=self.mlevObj, x0=np.array([theta2_init]), bounds=[(0, None)], approx_grad=True)
        self.theta2_hat = self.theta2_est[0]
        return(self)

    def getTheta1(self):
        self.theta1_hat = (1.0/self.n) * np.sum(self.yTildeSq/(self.theta2_hat/self.p*self.lbd+1.0))
        return(self.theta1_hat)

    def getMLEV(self):
        self.mlev_hat = self.eigen().getTheta2().getTheta1()
        return(self.mlev_hat)

## Unit Test
1. Low dimensional scenario: number of observations = 100; number of features = 10
2. Low dimensional scenario: number of observations = 10000; number of features = 1000
3. High dimensional scenario: number of observations = 200; number of features = 1000

In [47]:
# Form dataset 1:
import numpy as np
beta1 = np.random.normal(loc=0, scale=(10/10)**0.5, size=10)
X1 = np.random.rand(100, 10)
error1 = np.random.normal(loc=0, scale=10**0.5, size=100)
y1 = np.dot(X1, beta1) + error1
dataset1 = {'y': y1, 'X': X1}

# Form dataset 2:
import numpy as np
beta2 = np.random.normal(loc=0, scale=(10/1e3)**0.5, size=int(1e3))
X2 = np.random.rand(int(1e4), int(1e3))
error2 = np.random.normal(loc=0, scale=10**0.5, size=int(1e4))
y2 = np.dot(X2, beta2) + error2
dataset2 = {'y': y2, 'X': X2}

# Form dataset 3:
import numpy as np
beta3 = np.random.normal(loc=0, scale=(10/1e3)**0.5, size=int(1e3))
X3 = np.random.rand(200, 1000)
error3 = np.random.normal(loc=0, scale=10**0.5, size=200)
y3 = np.dot(X3, beta3) + error3
dataset3 = {'y': y3, 'X': X3}

# Show the true sample variance
print(np.var(error1))
print(np.var(error2))
print(np.var(error3))

9.6921439231
10.0146641758
10.6912475793


In [48]:
# Test Scenario 1
test1 = MLEV(dataset1)
test1.getMLEV()

9.8862540513587547

In [49]:
# Test Scenario 2
test2 = MLEV(dataset2)
test2.getMLEV()

10.026761934514186

In [50]:
# Test Scenario 3
test3 = MLEV(dataset3)
test3.getMLEV()

10.87882825161233

## Future Direction
- Accept different input data types <br>
  *e.g. matrix and pd.dataframe*
- Manipulation <br>
  *e.g. warning of repetitive observations*
- Help function
- *Statistical modeling